<a href="https://colab.research.google.com/github/KieranDingwall/Natural-Language-Processing-Coursework---Kieran-Dingwall/blob/main/NLP_Coursework_V2_(Colab_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Coursework

By Kieran Dingwall - Student Number: 2208619

### Section 1 - Dataset Exploring and Pre Processing

In [1]:
# Imports
import pandas as pd
import numpy as np
# import io

# Load Dataset
from google.colab import files, drive #import colab functionality
drive.mount("/content/drive")
path = '/content/drive/MyDrive/Uni/4th Year 1st Sem/Natural Language Processing/NLP Coursework/pricerunner_aggregate.csv'
product_data = pd.read_csv(path)
print(product_data)

Mounted at /content/drive
       Product ID                                      Product Title  \
0               1                    apple iphone 8 plus 64gb silver   
1               2                apple iphone 8 plus 64 gb spacegrau   
2               3  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
3               4                apple iphone 8 plus 64gb space grey   
4               5  apple iphone 8 plus gold 5.5 64gb 4g unlocked ...   
...           ...                                                ...   
35306       47350  smeg fab28 60cm retro style right hand hinge f...   
35307       47351  smeg fab28 60cm retro style left hand hinge fr...   
35308       47352  smeg fab28 60cm retro style left hand hinge fr...   
35309       47355     candy 60cm built under larder fridge cru160nek   
35310       47358           neff k4316x7gb built under larder fridge   

        Merchant ID   Cluster ID             Cluster Label   Category ID  \
0                 1            1 

In [2]:
# Remove unused columns from data
product_df = product_data.drop(product_data.columns[[0, 2, 3, 5]], axis=1)
print(product_df)

                                           Product Title  \
0                        apple iphone 8 plus 64gb silver   
1                    apple iphone 8 plus 64 gb spacegrau   
2      apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
3                    apple iphone 8 plus 64gb space grey   
4      apple iphone 8 plus gold 5.5 64gb 4g unlocked ...   
...                                                  ...   
35306  smeg fab28 60cm retro style right hand hinge f...   
35307  smeg fab28 60cm retro style left hand hinge fr...   
35308  smeg fab28 60cm retro style left hand hinge fr...   
35309     candy 60cm built under larder fridge cru160nek   
35310           neff k4316x7gb built under larder fridge   

                  Cluster Label  Category Label  
0      Apple iPhone 8 Plus 64GB   Mobile Phones  
1      Apple iPhone 8 Plus 64GB   Mobile Phones  
2      Apple iPhone 8 Plus 64GB   Mobile Phones  
3      Apple iPhone 8 Plus 64GB   Mobile Phones  
4      Apple iPhone 8 Plus 64

In [3]:
# Convert the feature columns into a numpy array
x = product_df["Product Title"].to_numpy()
y = product_df[" Category Label"].to_numpy()

# Testing outputs
#print(x)
#print(y)

In [4]:
# Checking Balance

print(product_df["Product Title"].value_counts())
print(product_df[" Category Label"].value_counts())

# Balance is good

Product Title
washing machine                                     90
built in fully integrated dishwasher                35
frost free fridge freezer                           34
washer dryer                                        24
american fridge freezer                             22
                                                    ..
sony hx400v compact camera with 50x optical zoom     1
sony hx400 20mp 50x zoom bridge camera               1
sony cyber shot hx400 digital camera                 1
sony cyber shot dsc hx400v digital camera            1
x t3 body only silver                                1
Name: count, Length: 30993, dtype: int64
 Category Label
Fridge Freezers     5501
Mobile Phones       4081
Washing Machines    4044
CPUs                3862
Fridges             3584
TVs                 3564
Dishwashers         3424
Digital Cameras     2697
Microwaves          2342
Freezers            2212
Name: count, dtype: int64


### Section 2 - Representation Learning

In [5]:
# Representation Learner
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_base = TfidfVectorizer(max_features=2000).fit_transform(x)
print(tfidf_base)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 247181 stored elements and shape (35311, 2000)>
  Coords	Values
  (0, 564)	0.47562446686894067
  (0, 1138)	0.45481272898552627
  (0, 1479)	0.4733224289764213
  (0, 381)	0.46740012928169644
  (0, 1641)	0.35274600698475417
  (1, 564)	0.35608862117257495
  (1, 1138)	0.3405073725965799
  (1, 1479)	0.3543651406619306
  (1, 380)	0.41707667122083275
  (1, 973)	0.35714545937478775
  (1, 1696)	0.5746086853970792
  (2, 564)	0.3331251324733054
  (2, 1138)	0.31854868945503123
  (2, 1479)	0.3315127959950069
  (2, 381)	0.32736484523179377
  (2, 50)	0.49365156932521237
  (2, 1642)	0.25020523620439966
  (2, 932)	0.19964760536977605
  (2, 1662)	0.28509409641708544
  (2, 1110)	0.1862759239990133
  (2, 998)	0.3291729646162801
  (3, 564)	0.39548184226606475
  (3, 1138)	0.37817687792502896
  (3, 1479)	0.39356769728379937
  (3, 381)	0.3886433038665723
  :	:
  (35307, 1664)	0.2440131092564437
  (35307, 944)	0.156914887522846
  (35307, 1097)	0.4146

### Section 3 - Algorithms

In [10]:
# Full Model Test
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# SKLearn Imports
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer # Import TfidfVectorizer

def mlp(input_dimension, num_classes):
  model = Sequential()
  model.add(Dense(128, input_shape=(input_dimension,), activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  return model

kf = StratifiedKFold(n_splits=5, shuffle=True)
xnp = np.array(x) #convert to numpy to standardise our arrays for the split
ynp = np.array(y)

# Initialize acc_score list
acc_score = []

# Map string labels to integers for to_categorical and StratifiedKFold
unique_labels = np.unique(ynp)
label_to_int = {label: i for i, label in enumerate(unique_labels)}
ynp_encoded = np.array([label_to_int[label] for label in ynp])

NUM_CLASSES = len(unique_labels) # This will be 10 based on your data
TFIDF_MAX_FEATURES = 2000 # Matching the previous TF-IDF setup

for train_index, test_index in kf.split(xnp, ynp_encoded):
  x_train, x_test = xnp[train_index], xnp[test_index]
  y_train_encoded, y_test_encoded = ynp_encoded[train_index], ynp_encoded[test_index]

  # Initialize a new TfidfVectorizer for each fold to prevent data leakage
  tfidf_vectorizer = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES)

  x_train_transformed = tfidf_vectorizer.fit_transform(x_train)
  x_train_dense = x_train_transformed.todense() # Convert to dense matrix for Keras

  x_test_transformed = tfidf_vectorizer.transform(x_test)
  x_test_dense = x_test_transformed.todense() # Convert to dense matrix for Keras

  # Instantiate a new mlp model for each fold and ensure correct input/output dimensions
  model = mlp(TFIDF_MAX_FEATURES, NUM_CLASSES)
  y_train_one_hot = to_categorical(y_train_encoded, NUM_CLASSES) # Convert y to one hot vectors
  y_test_one_hot = to_categorical(y_test_encoded, NUM_CLASSES)

  # Configure the model and start training (using categorical_crossentropy for multi-class)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x_train_dense, y_train_one_hot, epochs=10, batch_size=250, verbose=1, validation_split=0.2)

  # Test the model after training
  test_results = model.evaluate(x_test_dense, y_test_one_hot, verbose=1)
  print(f'Test results - Loss: {test_results[0]:.4f} - Accuracy: {test_results[1]*100:.2f}%')

  acc_score.append(test_results[1])

print("Accuracy:", np.mean(acc_score))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.4877 - loss: 1.7908 - val_accuracy: 0.3848 - val_loss: 3.7322
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9700 - loss: 0.1299 - val_accuracy: 0.3855 - val_loss: 4.5905
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9771 - loss: 0.0746 - val_accuracy: 0.4048 - val_loss: 4.9111
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9779 - loss: 0.0628 - val_accuracy: 0.3956 - val_loss: 5.2128
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9800 - loss: 0.0551 - val_accuracy: 0.4126 - val_loss: 5.2712
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9789 - loss: 0.0567 - val_accuracy: 0.4175 - val_loss: 5.4266
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9796 - loss: 0.0534 - val_accuracy: 0.3991 - val_loss: 5.5545
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9779 - loss: 0.0545 - val_accuracy: 0.4083 - val_loss

In [9]:
# SKLearn Imports
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC

acc_score = []

pipeline = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf_base', TfidfTransformer()),
  ('clf', SVC(kernel='linear', C=1))
])

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]

  pipeline.fit(x_train, y_train)
  predictions = pipeline.predict(x_test)
  acc = accuracy_score(predictions, y_test)
  acc_score.append(acc)

print("Accuracy:", np.mean(acc_score))

Accuracy: 0.9448893446059141


### Section 4 - Evaluation